# Exercise sheet 8

**Please turn in your exercises by January 16th.**

## Angel Ontiveros, Björn Plüster
## 16.01.2024

## Projected gradient method

### Task 1: Standard simplex

We considered projections onto the standard simplex in the lecture. Prove Lemma 10.4, i.e.,

**Lemma**

Under the assumption that $v_{1} \geq v_{2} \geq \cdots \geq v_{d}$, with $x^{*}(p)$ as
$$
x^{*}(p):=(v_{1}-\Theta_{p}, \ldots, v_{p}-\Theta_{p}, 0, \ldots, 0), \quad p \in\{1, \ldots, d\},
$$
and with
$$
p^{*}:=\max \left\{p \in\{1, \ldots, d\}: v_{p}-\frac{1}{p}\left(\sum_{i=1}^{p} v_{i}-1\right)>0\right\},
$$
it holds that
$$
\underset{x \in \Delta_{d}}{\text{argmin }}\|{x-v}\|^{2}=x^{*}(p^{*}) .
$$

**Proof:**

From the lecture we have that
$$
\Theta_{p} = \frac{1}{p}\left(\sum_{i=1}^{p} v_{i}-1\right),
$$
and thus
$$
p^{*} := \max \left\{p \in\{1, \ldots, d\}: v_{p}-\Theta_{p}>0\right\}.
$$


For $x^*(p^*)$ to be in the standard simplex $\Delta_{d}$, it must satisfy two conditions:
- Non-negativity: $x_i \geq 0$ for all $i$.
- Summation to 1: $\sum_{i=1}^d x_i = 1$.

$x^*(p^*)$ satisfies the non-negativity condition since for $i \leq p^*$, $x_i = v_i - \Theta_{p^*} \geq 0$ (by definition of $p^*$), and for $i > p^*$, $x_i = 0$.

For any $p$ the summation equals 1:
$$
\sum_{i=1}^{d} x_i^*(p) = \sum_{i=1}^{p} (v_i - \Theta_p) = \sum_{i=1}^{p} v_i - p \cdot \Theta_p = \sum_{i=1}^{p} v_i - \sum_{i=1}^{p} v_i + 1 = 1.
$$


We need to show that for any $x \in \Delta_d$, $\|x^*(p^*) - v\|^2 \leq \|x - v\|^2$.

Let's consider an arbitrary $x \in \Delta_d$. The squared Euclidean distance to $v$ is given by:
$$
\|x - v\|^2 = \sum_{i=1}^{d} (x_i - v_i)^2.
$$

If we choose a larger $p$ than $p^*$ we would violate the simplex constraint, since the values of x^*(p) would be negative for $i > p^*$.
On the other hand, if we choose a smaller $p$ than $p^*$, we would have $x_i = 0$ for $i > p$, and would have contributions to the distance from $(v_i)^2$ that are bigger,
since the components of $v$ are non-increasing. This $p^*$ is the largest $p$ that satisfies the simplex constraint, and minimizes the distance to $v$. $\square$




### Task 2: Probability

Suppose we are given a regression problem where we want to estimate a discrete probability distribution. It can be written in the following form:
$$
\begin{array}{lll}
\displaystyle\min_{w\in\mathbb{R}^d} & \left\|{Xw-y}\right\|_2^2\\
\text{s.t.}& \|{w}\|_1 = 1 \\
& w \geq 0
\end{array}
$$

The problem can be solved using projected gradient descent. Implement such an algorithm for solving this problem. Run it on the provided data.

In [18]:
import numpy as np
from sklearn.datasets import make_regression

def f(w):
    return np.linalg.norm(X @ w - y) ** 2 / len(X)

def g(w):
    return 2 * X.T @ (X @ w - y) / len(X)

X, y = make_regression(n_samples=1000, n_features=100, n_informative=40, random_state=0)
y /= 1000
X /= 10
x0 = np.zeros(100)


def projected_gradient_descent(w, lr, threshold, max_iter):
    iter = 0
    old_grad_norm = float('inf')
    while True:
        grad = g(w)
        w -= lr * grad
        w = np.maximum(w, 0)
        w /= np.sum(w)
        iter += 1
        grad_norm = np.linalg.norm(grad)
        grad_norm_diff = grad_norm - old_grad_norm
        if grad_norm_diff < threshold or iter > max_iter:
            break
        old_grad_norm = grad_norm
    return w, iter, grad_norm

In [21]:
lr = 1
threshold = 1e-6

w = np.zeros(X.shape[1])
max_iter = 1000
 
w, iter, grad_norm_diff = projected_gradient_descent(w, lr, threshold, max_iter)
print(f'num iterations: {iter}')
print(f'gradient norm diff: {grad_norm_diff}')
print(w)

num iterations: 1
gradient norm diff: 0.07684944367859972
[0.02062346 0.01327478 0.00249736 0.0102525  0.00056743 0.00246996
 0.         0.04321136 0.00393971 0.00779603 0.0358919  0.00438596
 0.00490427 0.00656058 0.         0.00825194 0.         0.00764423
 0.         0.04142642 0.00289623 0.01430778 0.         0.00703371
 0.         0.00080746 0.         0.0067681  0.04018039 0.04018011
 0.         0.         0.00459755 0.00659429 0.00841634 0.01793307
 0.         0.         0.00790067 0.00081205 0.00522957 0.
 0.         0.         0.         0.00955695 0.02050079 0.
 0.02258699 0.00496016 0.001968   0.01617429 0.00837852 0.00302537
 0.02794392 0.         0.03420394 0.         0.00406956 0.04113044
 0.04224314 0.01428942 0.         0.00152479 0.00116339 0.01704434
 0.0072764  0.00155718 0.00665287 0.01634742 0.         0.01309678
 0.00575232 0.         0.00012424 0.04679701 0.00166455 0.00967855
 0.         0.01199785 0.         0.         0.         0.03448475
 0.01110778 0.004215

## Proximal gradient method

### Task 3: Proximal operator

The LASSO is the following regularized optimization problem:
$$
\begin{array}{lll}
\displaystyle\min_{w\in\mathbb{R}^d} & \frac{1}{n} \left\|{Xw-y}\right\|_2^2 + \lambda \|{w}\|_1
\end{array}
$$

Write down the formula of the proximal operator for this case, and prove that the subgradient of the proximal operator contains 0 in that point.

$$
\text{prox}_{\lambda,\gamma}(z) = \dots
$$

**Proximal Operator Formula:**

For a given $\gamma > 0$, the proximal operator for the LASSO regularization term $\lambda \|w\|_1$ is defined as:
$$
\text{prox}_{\lambda, \gamma}(z) = \arg\min_w \left\{ \frac{1}{2\gamma}\|w - z\|_2^2 + \lambda \|w\|_1 \right\}
$$

**Proof that Subgradient Contains 0:**

   The subgradient of the L1 norm $\|w\|_1$ at a point $w$ is given by:
   $$
   \partial \|w\|_1 = \left\{ g \in \mathbb{R}^d : g_i \in \begin{cases}
   \{1\} & \text{if } w_i > 0\\
   [-1, 1] & \text{if } w_i = 0\\
   \{-1\} & \text{if } w_i < 0
   \end{cases} \right\}
   $$

### Task 4: LASSO (OPTIONAL)

Again, the LASSO is the following regularized optimization problem:
$$
\begin{array}{lll}
\displaystyle\min_{w\in\mathbb{R}^d} & \frac{1}{n} \left\|{Xw-y}\right\|_2^2 + \lambda \|{w}\|_1
\end{array}
$$

It is a regression method that provides sparse solutions/regressors. It is used when the data is very high-dimensional and only a few data points are given. This is usually the case when working with genome data. Here, one tries to predict which genes are responsible for a disease. Usually, we have only a few hundred data points (patients, the rows of the data matrix $X$) and a few thousand genes (the columns of the data matrix $X$). The label vector $y$ contains the label either $0$ for not having the disease and $1$ for having the disease. Without the regularization term infinitely many solutions exist. However, one usually assumes that only a few genes/features are responsible for causing the disease. Hence, we can use a $\|{w}\|_1$ regularizer to enforce sparse solutions.

Implement a proximal gradient descent algorithm for solving the LASSO problem. Run it on the provided data.

In [ ]:
from sklearn.datasets import make_regression

lam = .5

def f(w):
    return np.linalg.norm(X @ w - y) ** 2 / len(X) + lam * np.linalg.norm(w, 1)

def g(w):
    return 2 * X.T @ (X @ w - y) / len(X) + lam * np.sign(w)

def proximal_operator(w):
    ...

X, y = make_regression(n_samples=30, n_features=100, n_informative=2, random_state=0)
x0 = np.zeros(100)

...